# Power Transformer 101
## By: Patrick L. Cavins 

### This notebook was developed to be used with lighting talk 

## What? 

### Powertransformer is part of sklearn preprocessing module


### The individual transformers can be found inside the SciPy stats module

- The key part of this technique is found right in the name **TRANSFORM** 
- Two data transformers are inside this module 
    - Box-Cox Transformation (will take in **ONLY** positive values) 
    - Yeo Johnson Transformation (can take in **BOTH** positive and negative values) 
    - What about 0? 
        - She ain't no hero here... 
    
    
![alt text](boxcox_beforeafter.png "Title")    

# The Math 

## Lambda can be between -5 and 5

## The Confidence Interval is important! 

![alt text](math_boxcox.png "Title")

In [4]:
#Libraries needed for this demonstration

import pandas as pd 
import numpy as np 
from sklearn.preprocessing import PowerTransformer
from scipy import stats
import seaborn as sns 
import matplotlib.pyplot as plt

%matplotlib inline 

## A Quick Demo

In [6]:
# Import the Data, in this case AMES Housing Data. We are only working with one-column 
df = pd.read_csv('../data/train.csv')

df['Garage Area'].head(10)


0    475.0
1    559.0
2    246.0
3    400.0
4    484.0
5    578.0
6    525.0
7    531.0
8    420.0
9    504.0
Name: Garage Area, dtype: float64

## Import the data and some quick EDA 

In [ ]:
#Load the Data
df = pd.read_csv('./train.csv')

#Checking for Nulls / EDA 
nulls = df['Garage Area'].isnull().sum()
print (f'The numbers of nulls: {nulls}')

#Replace the Null with 0 
df['Garage Area'] = df['Garage Area'].replace(np.nan, 0)
nulls = df['Garage Area'].isnull().sum()
print (f'The numbers of nulls: {nulls}')

#Drop Zeros from the pd.series (if zero we can assume that a garage is not around) 
garage_area = df['Garage Area']
garage_area = garage_area[garage_area != 0]

garage_area = np.asarray(df['Garage Area'])

In [ ]:
# We can see that this data is screwed, and does not represent a normal distribution 

plt.figure(figsize=(8,5))
sns.kdeplot(garage_area, shade=True);

In [ ]:
# Values in the Box-Cox Transformation
# xt = (x**lambda - 1) / lambda 

# define the set of lambda's that we want to search over
lmbda = np.linspace(start = .1, stop = 1.0, num=4) 

xt = []
lambda_list = []
for i in lmbda: 
    #x = the input data 
    x = garage_area 
    #box-cox transformation 
    transform = (x**i - 1) / i
    #appending the list 
    xt.append(transform)

In [ ]:
print ('These are the tranformed values, Box-Cox Transformation for a given Lambda')
print ('--'*50)
print (xt[1]) # we can grab the individual values 
print ('--'*50)
print ('These are the lambdas which we used in this transformation')
print (lmbda) # this is the list of lambda's we generated 

In [ ]:
print (lmbda)

## The lambda has **HUGE** effect on the distrubtion of the data 

In [ ]:
sns.kdeplot(garage_area, shade=True, label="Raw")
sns.kdeplot(xt[0], shade=True, label= f"BC_Tranform(Lambda Value: {round(lmbda[0],3)}")
sns.kdeplot(xt[1], shade=True, label= f"BC_Tranform(Lambda Value: {round(lmbda[1],3)}")
sns.kdeplot(xt[2], shade=True, label= f"BC_Tranform(Lambda Value: {round(lmbda[2],3)}")
sns.kdeplot(xt[3], shade=True, label= f"BC_Tranform(Lambda Value: {round(lmbda[3],3)}")

# set the legend location
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# control x and y limits
plt.ylim(0, 0.06)
plt.xlim(0, 300)


## The trick of Box-Cox transformation is to find lambda value...

However in practice this is quite affordable. The following function returns the transformed variable, lambda value, confidence interval for lambda according to certain alpha level (default = 0.05). 

- garage_area_xt: transformed variable
- maxlog: lambda
- interval: confidence interval

- http://dataunderthehood.com/2018/01/15/box-cox-transformation-with-python/

In [ ]:
garage_area_xt, maxlog, interval = stats.boxcox(garage_area, alpha=0.95)

print (maxlog)
print (interval)
print (garage_area_xt)

## The change is so dramatic you cannot visualize it on a single graph, easily

In [ ]:
# Changes in the KDE... 
sns.kdeplot(garage_area, shade=True, label="Raw")
sns.kdeplot(garage_area_xt, shade=True, label="Transformed")
plt.title('Comparing the Raw Input to the Transformed Output')

## Data normalization, ZScore or MinMax Scaler

In [ ]:
## Z-Score (garage area)

garage_area_std = np.std(garage_area)

print (garage_area_std)

garage_area_Z = ((garage_area -garage_area.mean()) / (garage_area_std)) 

# plot a histogram
plt.hist(garage_area_Z);

In [ ]:
## Z-Score (garage area_xt)

garage_area_xt_std = np.std(garage_area_xt)

print (garage_area_xt_std)

garage_area_xt_Z = ((garage_area_xt - garage_area_xt.mean()) / (garage_area_xt_std))

# plot a histogram 
plt.hist(garage_area_xt_Z);

In [ ]:
# Changes in the KDE!  
plt.figure(figsize=(13,8))
sns.set_context('poster')
sns.kdeplot(garage_area_Z, shade=True, color='navy', label="Raw (normalized)")
sns.kdeplot(garage_area_xt_Z, shade=True, color='gold', label="Transformed (normalized)")

## Using the Sklearn Module

In [ ]:
# Let's use two features...

print (df['Lot Area'].min())

In [ ]:
plt.figure(figsize=(8,5))
sns.kdeplot(df['Lot Area'], shade=True)

In [ ]:
#Features 
x = df[['Garage Area', 'Lot Area']]

## Instantiate PowerTransformer 


### Parameters 

**method** : str, (default=’yeo-johnson’)

**standardize** : boolean, default=True
Set to True to apply zero-mean, unit-variance normalization to the transformed output.

### Attributes

**lambdas_**: array of float, shape (n_features,)
The parameters of the power transformation for the selected features.

In [ ]:
pt = PowerTransformer(method='yeo-johnson', standardize=True,) 

#Fit the data to the powertransformer
skl_yj = pt.fit(x)

In [ ]:
#Here are the lambdas 
skl_yj.lambdas_

In [ ]:
#Transform 
skl_yj = pt.transform(x)

In [ ]:
lot_area = np.asarray(df['Lot Area'])

In [ ]:
plt.figure(figsize=(6,6))
sns.set_context('poster')
sns.kdeplot(garage_area, shade=True, color='navy', label="Garage Area")
sns.kdeplot(lot_area, shade=True, color='gold', label="Lot Area")

# control x and y limits
# plt.ylim(0, 0.06)
# plt.xlim(0, 20_000)

In [ ]:
plt.figure(figsize=(8,4))
plt.hist(skl_boxcox, bins=15);

## Good Tidbits:

"Box-cox transformation is a statistical technique used to remove heteroscedasticity of a variable and also make it look like more normally distributed, which represents a big deal for statisticians and economists regarding normality and homoscedasticity assumptions for linear models."

http://dataunderthehood.com/2018/01/15/box-cox-transformation-with-python/



"For example, the data may have a skew, meaning that the bell in the bell shape may be pushed one way or another."

https://machinelearningmastery.com/how-to-transform-data-to-fit-the-normal-distribution/



"But, generally the answer is that for most meaningful analysis, you need the same 𝜆 value for all datasets. The reason is that the Box-Cox transformation **not only changes the scale of the data, it also changes the unit of measurement**. "

https://stats.stackexchange.com/questions/243975/skewness-transformation-for-one-but-not-the-other-variable/243984#243984

https://stats.stackexchange.com/questions/243975/skewness-transformation-for-one-but-not-the-other-variable/243984#243984

#### References: 

http://www.kmdatascience.com/2017/07/box-cox-transformations-in-python.html

    